In [1]:
subject_id: int = 3
low_cut_hz: float = 4.
high_cut_hz: float = 38.
n_epochs: int = 3

In [ ]:
""" 
Baseline implementation by Robin
"""

# %% [markdown]
# ## notebook setup, not necessary for code to run

# %%

# %% [markdown]
# ## Hyperparameters

# %%
n_recordings_to_load = 300
target_name = 'pathological'
n_max_minutes = 3
sfreq = 100
n_minutes = 2
input_window_samples = 1200
n_epochs = 50
batch_size = 64
# This was from High-Gamma dataset optimization:
#lr = 1 * 0.01
#weight_decay = 0.5 * 0.001
lr = 5e-3
weight_decay = 5e-4

seed = 20210325  # random seed to make results reproducible

# atm window stride determined automatically as n_preds_per_input, could also parametrize it 

# %% [markdown]
# ## Set random seeds for reproducibility

# %%
from braindecode.util import set_random_seeds
cuda = False
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

# %% [markdown]
# ## Model definition

# %%
import torch as th

th.backends.cudnn.benchmark = True

# %%
import torch
from braindecode.models import Deep4Net


n_classes = 2
# Extract number of chans from dataset
n_chans = 21

model = Deep4Net(
    in_chans=n_chans,
    n_classes=n_classes, 
    input_window_samples=None,
    final_conv_length=2,
    stride_before_pool=True,
)

# Send model to GPU
if cuda:
    model.cuda()
from braindecode.models.util import to_dense_prediction_model, get_output_shape
to_dense_prediction_model(model)


# %% [markdown]
# ## Data Loading

# %%

from braindecode.datasets.tuh import TUHAbnormal
data_path = '/home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/'
dataset = TUHAbnormal(
    path=data_path,
    #recording_ids=range(n_recordings_to_load),  # loads the n chronologically first recordings
    target_name=target_name,  # age, gender, pathology
    preload=False,
    add_physician_reports=False,
)

# %%
from braindecode.datasets import BaseConcatDataset
#dataset = BaseConcatDataset(dataset.datasets[:n_recordings_to_load])

# %% [markdown]
# ## Data Preprocessing

# %%

from braindecode.preprocessing import preprocess, Preprocessor
import numpy as np
from copy import deepcopy


whole_train_set = dataset.split('train')['True']

ar_ch_names = sorted([
    'EEG A1-REF', 'EEG A2-REF',
    'EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF',
    'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF',
    'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF',
    'EEG T6-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF'])

preprocessors = [
    Preprocessor(fn='pick_channels', ch_names=ar_ch_names, ordered=True),
    Preprocessor('crop', tmin=0, tmax=n_max_minutes*60, include_tmax=True),
    Preprocessor(fn=lambda x: np.clip(x, -800,800), apply_on_array=True),
    Preprocessor('set_eeg_reference', ref_channels='average'),
    # convert from volt to microvolt, directly modifying the numpy array
    Preprocessor(fn=lambda x: x * 1e6, apply_on_array=True),
    Preprocessor(fn=lambda x: x / 30, apply_on_array=True), # this seemed best
    Preprocessor(fn='resample', sfreq=sfreq),
]
# Preprocess the data
preprocess(whole_train_set, preprocessors)

# %% [markdown]
# ## Data Splitting

# %%
import numpy as np

from braindecode.datasets.base import BaseConcatDataset


subject_datasets = whole_train_set.split('subject')
n_subjects = len(subject_datasets)

n_split = int(np.round(n_subjects * 0.75))
keys = list(subject_datasets.keys())
train_sets = [d for i in range(n_split) for d in subject_datasets[keys[i]].datasets]
train_set = BaseConcatDataset(train_sets)
valid_sets = [d for i in range(n_split, n_subjects) for d in subject_datasets[keys[i]].datasets]
valid_set = BaseConcatDataset(valid_sets)

# %% [markdown]
# ## Data Compute Window Creation

# %%
import pandas as pd

from braindecode.models.util import to_dense_prediction_model, get_output_shape

n_preds_per_input = get_output_shape(model, n_chans, input_window_samples)[2]

# %%
from braindecode.datautil.windowers import create_fixed_length_windows


window_train_set = create_fixed_length_windows(
    train_set,
    start_offset_samples=60*sfreq,
    stop_offset_samples=60*sfreq+n_minutes*60*sfreq,
    preload=True,
    window_size_samples=input_window_samples,
    window_stride_samples=n_preds_per_input,
    drop_last_window=True,
)

window_valid_set = create_fixed_length_windows(
    valid_set,
    start_offset_samples=60*sfreq,
    stop_offset_samples=60*sfreq+n_minutes*60*sfreq,
    preload=True,
    window_size_samples=input_window_samples,
    window_stride_samples=n_preds_per_input,
    drop_last_window=False,
)

# %% [markdown]
# ## Initialize Data Loaders

# %%
num_workers = 0

# %%
train_loader = th.utils.data.DataLoader(
    window_train_set,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True)
train_det_loader = th.utils.data.DataLoader(
    window_train_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False)
valid_loader = th.utils.data.DataLoader(
    window_valid_set,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    drop_last=False)

print(len(valid_loader.dataset))
# %% [markdown]
# ## Initialize Optimizer and Scheduler

# %%
optim = th.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optim, T_max=n_epochs,)

# %% [markdown]
# ## Run Training

# %%
from tqdm.autonotebook import tqdm, trange
from braindecode.training.scoring import trial_preds_from_window_preds
import pandas as pd


epochs_df = pd.DataFrame()

for i_epoch in trange(n_epochs):
    if i_epoch > 0:
        for X,y,i in tqdm(train_loader):
            #print(X.shape)
            #print(i)
            X = X.cuda()
            y = y.type(th.LongTensor).cuda()
            out = model(X)
            #print(out.shape)
            cross_ent = th.nn.functional.cross_entropy(th.mean(out, dim=(2)), y)
            optim.zero_grad()
            cross_ent.backward()
            optim.step()
            optim.zero_grad()
        scheduler.step()

    epoch_results = {}
    epoch_results['epoch'] = i_epoch
    for name, loader in {'train': train_det_loader, 'valid': valid_loader}.items():
        print(f"Epoch {i_epoch:d}")
        all_preds = []
        all_is = []
        all_ys = []
        with th.no_grad():
            for X,y,i in tqdm(loader):
                preds = model(X.cuda())
                all_preds.extend(preds.cpu().numpy())
                all_is.extend(i)
                all_ys.extend(y.cpu().numpy())
        all_preds = np.array(all_preds)
        #print(all_preds.shape) # (nb_crops, 2, nb_pred_per_crop)
        all_ys = np.array(all_ys)
        crop_preds = np.mean(all_preds, axis=(2)).argmax(axis=1)
        acc_cropwise = np.mean((crop_preds == all_ys))
        trial_ys = all_ys[np.diff(th.cat(all_is[0::3]), prepend=[np.inf]) != 1]
        #print(all_preds.shape, th.cat(all_is[0::3]).shape)
        preds_per_trial = trial_preds_from_window_preds(all_preds, th.cat(all_is[0::3]), 
                                                        th.cat(all_is[2::3]),)
        trial_preds = np.array([p.mean(axis=1).argmax(axis=0) for p in preds_per_trial])
        acc_trial = np.mean(trial_preds  == trial_ys)
        print(f"{name.capitalize()} Crop Acc:                {acc_cropwise:.1%}")
        print(f"{name.capitalize()} Recording Acc:           {acc_trial:.1%}")

        results = dict(acc_cropwise=acc_cropwise,
                       acc_trial=acc_trial,)
        #add 'train', 'valid' prefix
        results = {name + '_' + key: value for key, value in results.items()}
        epoch_results = {**epoch_results, **results}
        """
    epochs_df = epochs_df.append(
        epoch_results,
        ignore_index=True)
        """
# %%
epochs_df

# %%





Tensorflow not install, you could not use those pipelines
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/041/00004196/s003_2009_09_03/00004196_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/058/00005864/s001_2009_09_03/00005864_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/009/00000929/s003_2009_09_04/00000929_s003_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/058/00005851/s001_2009_09_04/00005851_s001_t001.edf...
EDF file detected
Setting channel info struc

/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriat

Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/061/00006183/s001_2009_09_21/00006183_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/061/00006140/s001_2009_09_22/00006140_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/061/00006146/s001_2009_09_22/00006146_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/061/00006156/s001_2009_09_23/00006156_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info st

/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriat

Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/022/00002272/s002_2009_10_12/00002272_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/060/00006005/s001_2009_10_13/00006005_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/061/00006163/s001_2009_10_13/00006163_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/061/00006186/s001_2009_10_13/00006186_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF param

...

Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/093/00009345/s001_2012_05_29/00009345_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/abnormal/01_tcp_ar/085/00008569/s002_2012_05_30/00008569_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/abnormal/01_tcp_ar/088/00008880/s001_2012_05_30/00008880_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/090/00009019/s001_2012_05_30/00009019_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting E

/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriat

Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/eval/normal/01_tcp_ar/093/00009356/s001_2012_06_01/00009356_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/046/00004628/s003_2012_06_04/00004628_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/abnormal/01_tcp_ar/067/00006774/s002_2012_06_04/00006774_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/abnormal/01_tcp_ar/086/00008648/s002_2012_06_04/00008648_s002_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting ED

/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriate target is added to description.
  warnings.warn(f"'{name}' not in description. '__getitem__'"
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datasets/base.py:131: UserWarning: 'pathological' not in description. '__getitem__'will fail unless an appropriat

Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/087/00008758/s001_2012_06_06/00008758_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/091/00009115/s001_2012_06_06/00009115_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/normal/01_tcp_ar/017/00001754/s002_2012_06_07/00001754_s002_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /home/jovyan/mne_data/TUH/tuh_eeg_abnormal/v2.0.0/edf/train/abnormal/01_tcp_ar/071/00007176/s003_2012_06_07/00007176_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF

In [2]:
import pooch
import pooch.utils

In [3]:
def download_patch(*args,fname=None,path=None, **kwargs):
    path = pooch.utils.cache_location(path, env=None, version=None)
    full_path = path.resolve() / fname
    print('Not downloading to avoid side effects')
    return str(full_path)

pooch.retrieve = download_patch

In [6]:
from braindecode.datasets.moabb import MOABBDataset

dataset = MOABBDataset(dataset_name="Schirrmeister2017", subject_ids=[subject_id])

Tensorflow not install, you could not use those pipelines


/opt/conda/envs/demo-env/lib/python3.9/site-packages/moabb/datasets/download.py:54: RuntimeWarning: Setting non-standard config type: "MNE_DATASETS_SCHIRRMEISTER2017_PATH"
  set_config(key, get_config("MNE_DATA"))


Not downloading to avoid side effects
Not downloading to avoid side effects
Extracting EDF parameters from /home/jovyan/mne_data/MNE-schirrmeister2017-data/robintibor/high-gamma-dataset/raw/master/data/train/3.edf...
EDF file detected
Channel 'EEG Fp1' recognized as type EEG (renamed to 'Fp1').
Channel 'EEG Fp2' recognized as type EEG (renamed to 'Fp2').
Channel 'EEG Fpz' recognized as type EEG (renamed to 'Fpz').
Channel 'EEG F7' recognized as type EEG (renamed to 'F7').
Channel 'EEG F3' recognized as type EEG (renamed to 'F3').
Channel 'EEG Fz' recognized as type EEG (renamed to 'Fz').
Channel 'EEG F4' recognized as type EEG (renamed to 'F4').
Channel 'EEG F8' recognized as type EEG (renamed to 'F8').
Channel 'EEG FC5' recognized as type EEG (renamed to 'FC5').
Channel 'EEG FC1' recognized as type EEG (renamed to 'FC1').
Channel 'EEG FC2' recognized as type EEG (renamed to 'FC2').
Channel 'EEG FC6' recognized as type EEG (renamed to 'FC6').
Channel 'EEG M1' recognized as type EEG (re

In [ ]:
from braindecode.datautil.preprocess import (
    exponential_moving_standardize, preprocess, Preprocessor)

low_cut_hz = low_cut_hz
high_cut_hz = high_cut_hz

factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(lambda x: x * 1e6),  # Convert from V to uV
    Preprocessor('filter', l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(exponential_moving_standardize,  # Exponential moving standardization
                 factor_new=factor_new, init_block_size=init_block_size)
]

preprocess(dataset, preprocessors)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/datautil/preprocess.py:10: UserWarning: datautil.preprocess module is deprecated and is now under preprocessing.preprocess, please use from import braindecode.preprocessing.preprocess
  warn('datautil.preprocess module is deprecated and is now under '
/opt/conda/envs/demo-env/lib/python3.9/site-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


In [ ]:
from braindecode.datautil.windowers import create_windows_from_events

trial_start_offset_seconds = -0.5
sfreq = dataset.datasets[0].raw.info['sfreq']
assert all([ds.raw.info['sfreq'] == sfreq for ds in dataset.datasets])
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

splitted = windows_dataset.split('run')
train_set = splitted['train']
valid_set = splitted['test']

In [ ]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = 'cuda' if cuda else 'cpu'
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220  # random seed to make results reproducible
# Set random seed to be able to reproduce results
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
# Extract number of chans and time steps from dataset
n_chans = train_set[0][0].shape[0]
input_window_samples = train_set[0][0].shape[1]

model = ShallowFBCSPNet(
    n_chans,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length='auto',
)

# Send model to GPU
if cuda:
    print("Using GPU")
    model.cuda()
else:
    print("Using CPU")


In [ ]:
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

from braindecode import EEGClassifier
# These values we found good for shallow network:
lr = 0.0625 * 0.01
weight_decay = 0

# For deep4 they should be:
# lr = 1 * 0.01
# weight_decay = 0.5 * 0.001

batch_size = 64

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=predefined_split(valid_set),  # using valid_set for validation
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy", ("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
    ],
    device=device,
)

clf.fit(train_set, y=None, epochs=n_epochs)

In [ ]:
import pandas as pd

# Extract loss and accuracy values for plotting from history object
results_columns = ['train_loss', 'valid_loss', 'train_accuracy', 'valid_accuracy']
df = pd.DataFrame(clf.history[:, results_columns], columns=results_columns,
                  index=clf.history[:, 'epoch'])
df

In [ ]:
import matplotlib.pyplot as mp
mp.figure()
df.plot()

In [ ]:
import os
job_name = os.environ.get('JOB_NAME', None)

if job_name is not None:  
    model_file = f"{job_name}.model.pt"
    model_path = os.path.join(os.getcwd(), model_file)
else:
    model_path = os.path.join(os.getcwd(), 'model.pt')

if not os.path.exists(model_path):
    torch.save(model.state_dict(), model_path)
    print(f"saved model to {model_path}")
else:
    print(f"failed to save model to path {model_path}: file exists")